В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [44]:
n1 = 34
n2 = 16
a = 10.
b = 4.

In [46]:
p1 = a/n1
p2 = b/n2
p = (p1*n1 + p2*n2)/(n1 + n2)
Z = (p1 - p2)/np.sqrt((p*(1-p)*(1./n1 + 1./n2)))


In [9]:
#  функция для расчета p-value
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [48]:
p_value = proportions_diff_z_test(Z, alternative = 'greater')
round(p_value,4)

0.3729

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам 
X 
1
​	 ,X 
2
​	 ,X 
3
​	 ;
логистическая регрессия по признакам 
X 
4
​	 ,X 
5
​	 ,X 
6
​	 .
Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 
5.5×10 
−8
 , нужно ввести 8).

In [14]:
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [62]:
# вычисление z-статистики для 2  зависимых выборок
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [63]:
#  функция для расчета p-value для разности долей
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [64]:
data = pd.read_csv('banknotes.txt', delimiter= '\t')
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [65]:
X = data.drop('real', 1)
y = data['real']
y = y.reset_index(drop=True)

In [66]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state = 1)

Настройте два классификатора поддельности банкнот: логистическая регрессия по признакам X 1​ ,X 2​ ,X 3​

In [67]:
X_1_train = X_train.drop(columns=['X6', 'X4','X5'])
X_1_test = X_test.drop(columns=['X6', 'X4','X5'])
X_2_train = X_train.drop(columns=['X1', 'X2','X3'])
X_2_test = X_test.drop(columns=['X1', 'X2','X3'])

In [68]:
y_test =  y_test.to_numpy()

In [69]:
from sklearn.linear_model import LogisticRegression
model_1 = LogisticRegression(random_state = 1).fit(X_1_train, y_train)
y_regr1 = model_1.predict(X_1_test)
model_2 = LogisticRegression(random_state = 1).fit(X_2_train, y_train)
y_regr2 = model_2.predict(X_2_test)

In [70]:
y_regr1

array([1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0])

In [71]:
y_regr2

array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0])

In [72]:
y_test

array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0])

In [81]:
y_err1.reshape(1,len(y_test))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
        0., 0.]])

In [102]:
Z = proportions_diff_z_stat_rel(y_regr1 != y_test, y_regr2 != y_test)
p_val = proportions_diff_z_test(Z, alternative = 'two-sided')
p_val

0.0009231887438696518

In [103]:
import math

math.log10( p_val)

-3.0347094993628447

In [124]:
# построение доверительного интервала
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [125]:
a,b = proportions_diff_confint_rel(y_regr1 != y_test, y_regr2 != y_test, alpha = 0.05)
round(a,4)

0.0735

Eжегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [145]:
n = 100
se = 100
mu0 = 525
mu = 541.4
mu6 = 541.5

In [157]:
T = (mu6-mu0)/(se/np.sqrt(n))
T

1.65

In [158]:
from scipy.stats import norm
alpha = 0.05
p = 1 - norm.cdf(T)
round(p,4)


0.0495